In [ ]:
#import numpy
#numpy.version.version

In [ ]:
# память JVM для Hail
hl.init(sc, spark_conf={'spark.driver.memory': '8g', 'spark.executor.memory': '8g'}) 

In [ ]:
with gzip.open('/home/julia/Downloads/gnomADru/vcf/000007000070.vcf.gz', 'rt') as f:
    for l in f.readlines()[2500:5000]:
        print(l.strip())

In [ ]:
#работает только с VDS!
combiner = hl.vds.new_combiner(
    output_path='/home/julia/Downloads/gnomADru/vcf/test.vds',
    temp_path='/tmp',
    gvcf_paths=vcf_files,
    use_genome_default_intervals=True,
)

combiner.run()

In [ ]:
# Импорт всех VCF файлов в один MatrixTable - работает только для одного сэмпла! если нужно объединять разные образцы - не прокатит,
# потребуется комбинировать по одному...печаль
mt_list = [hl.import_vcf(vcf, reference_genome='GRCh38', force_bgz=True) for vcf in vcf_files]

In [ ]:
#Вытащить одну хромосому
prefix = '/home/julia/Downloads/gnomADru/chr22/'

for vcf in vcf_files:
    df = hl.import_vcf(vcf, force_bgz=True)
    meta = hl.methods.get_vcf_metadata(vcf)
    dn = df.filter_rows(df.locus.contig == 'chr22')
    basename = os.path.basename(vcf).split('.')[0]
    path = prefix + basename + '.vcf.bgz'
    # hl.export_vcf(dn, path, tabix=True, parallel='separate_header')
    hl.export_vcf(dn, path, tabix=True, metadata=meta)
    # df.show()
    # dn.show()
    
    # print(path)
    
#альтернатива - не проверяла
mt = mt.filter_rows(mt.locus.contig == '22')

In [ ]:
#метадата
hl.methods.get_vcf_metadata('/home/julia/Downloads/gnomADru/vcf/000007000020.vcf.gz')

In [ ]:
# импорт один файл
df = hl.import_vcf('/home/julia/Downloads/gnomADru/chr22/000007000020.vcf.bgz')
df.show()

In [ ]:
#пока непонятно, что такое и работает ли
all_variants = [mt.rows().select() for mt in mts]
combined_variants = all_variants[0]
for variants in all_variants[1:]:
    combined_variants = combined_variants.union(variants)
combined_new = mts[0].union_cols(*mts[1:], row_join_type='outer')

In [ ]:
# выбрать определённые столбцы
combined_mt.select_rows(
    rsid=combined_mt.rsid,
    chrom=combined_mt.locus.contig,
    pos=combined_mt.locus.position
).show(n_rows=10)

In [ ]:
#deep seek - на случай, если структура окажется неправильной
# hl.init()

# Список путей к VCF файлам
vcf_paths = [
    "/home/julia/Downloads/gnomADru/chr22/000007000040.vcf.bgz",
    "/home/julia/Downloads/gnomADru/chr22/000007000030.vcf.bgz",
    # добавьте остальные файлы
]

# Загрузка каждого VCF файла отдельно
mts = []
for path in vcf_paths:
    mt = hl.import_vcf(path, force_bgz=True, array_elements_required=False)
    # Фильтрация по хромосоме (если необходимо)
    mt = mt.filter_rows(mt.locus.contig == '22')
    # Оставляем только общие поля (если необходимо)
    mt = mt.select_entries('GT')
    # Переименование образцов (если необходимо)
    mt = mt.key_cols_by(sample_id=path.split('/')[-1] + "_" + mt.s)
    mts.append(mt)

# Объединение MatrixTable по колонкам (образцам) с внешним объединением по строкам
combined_mt = mts[0]
for mt in mts[1:]:
    combined_mt = combined_mt.union_cols(mt, row_join_type='outer')

# Проверка количества строк
print("Объединенная таблица:", combined_mt.count())  # Количество строк должно быть равно общему числу уникальных вариантов

# Пример вывода статистики
combined_mt.describe()

# Остановка Hail
hl.stop()

In [ ]:
# типы данных, содержащиеся в Matrix
print(filtered_mt.row.dtype)

In [ ]:
#найти конкретный вариант
test_intervals = ['chr22:10516121-10516122'] 
mint = hl.filter_intervals(
    mts[1],
    [hl.parse_locus_interval(x,)
     for x in test_intervals])
mint.GT.show()

In [ ]:
%%time

#посмотреть, сколько вариантов с нулевой частотой - убедиться, что данные по частоте есть

count = freq_mt_all.aggregate_rows(
    hl.agg.count_where(freq_mt_all.allele_frequencies[0] > 0)
)
print(f"Вариантов с AF[0] > 0: {count}")

In [ ]:
# фильтрация по глубине прочтения DP

filtered_mt = mt.filter_rows(
    (mt.info.DP >= 20) &          # Общая глубина по варианту
    (mt.entry.DP >= 10)           # Глубина у отдельного образца
)

In [ ]:
# отделить структурные варианты

mt = mt.annotate_rows(
    variant_type = hl.cond(
        (hl.len(mt.alleles[0]) - hl.len(mt.alleles[1])).abs() >= 50,
        "SV",
        "indel"
    )
)
# Фильтрация SVs
svs = mt.filter_rows(mt.variant_type == "SV")

In [ ]:
# среднее DP по вариантам (агрегированное по всем образцам)
freq_mt_all = freq_mt_all.annotate_rows(
    mean_DP = hl.agg.mean(freq_mt_all.DP)
)

# Строим гистограмму
freq_mt_all.aggregate_rows(
    hl.agg.hist(freq_mt_all.mean_DP, 0, 100, 20)
).show()

In [ ]:
# фильтрация редких (MAF) > 1%
filtered = freq_mt_all.filter_rows(
    hl.min(freq_mt_all.allele_frequencies) > 0.01
)

In [ ]:
# поиск частых, с частотой основного аллеля > 95%
common_vars = freq_mt_all.filter_rows(
    hl.max(freq_mt_all.allele_frequencies) > 0.95
)

In [ ]:
# запуск на кластере (bash) (например, AWS EMR, Google Dataproc, YARN)

spark-submit \
    --master yarn \
    --executor-memory 16G \
    --driver-memory 8G \
    my_hail_script.py

In [ ]:
# Для YARN (Hadoop-кластер)
hl.init(
    spark_conf={
        "spark.master": "yarn",
        "spark.executor.memory": "16G",
        "spark.driver.memory": "8G",
    }
)

# Для AWS EMR (пример)
hl.init(
    spark_conf={
        "spark.master": "spark://<master-IP>:7077",
        "spark.executor.instances": "10",
    }
)

In [2]:
# Аутосомы (1-22)
autosomes = [str(x) for x in range(1, 23)]

# Половые хромосомы и митохондриальная ДНК
sex_and_mito = ["X", "Y", "MT"]

# Все хромосомы GRCh38 (стандартный порядок)
all_chromosomes = autosomes + sex_and_mito

# Альтернативные варианты (например, "chr1" вместо "1")
all_chromosomes_with_prefix = ["chr" + x for x in all_chromosomes]
print(all_chromosomes_with_prefix)

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrMT']


In [ ]:
zcat 000007000020.vcf.gz | grep -m 10 "##FORMAT"

In [ ]:
hl.summarize_variants(combined_mt_all)

In [ ]:
# учет мультиаллельных вариантов - не факт, что работает

mt = hl.split_multi_hts(mt)  # Разделяет мультиаллельные варианты
mt = mt.annotate_rows(
    AF = hl.agg.call_stats(mt.GT, mt.alleles).AF[1]  # Частота первого альтернативного
)

In [ ]:
# проверка правильности гемизиготности - РАБОТАЕТ

male_x_entries = mt_normalized.filter_entries(
        (~mt_normalized.is_female) & 
        (mt_normalized.locus.contig == "chrX")
    )
    
    # Агрегируем результаты
    result = male_x_entries.aggregate_entries(
        hl.struct(
            total=hl.agg.count(),
            hom_ref=hl.agg.count_where(male_x_entries.GT.is_hom_ref()),
            hom_var=hl.agg.count_where(male_x_entries.GT.is_hom_var()),
            het=hl.agg.count_where(male_x_entries.GT.is_het()),
            missing=hl.agg.count_where(hl.is_missing(male_x_entries.GT))
        )
    )
    
    print(f"""
    Результаты нормализации для мужчин на X-хромосоме:
    Всего записей: {result.total}
    Гомозиготные референсные (0/0→0): {result.hom_ref}
    Гомозиготные альтернативные (1/1→1): {result.hom_var}
    Гетерозиготные (0/1→NA): {result.het}
    Пропущенные значения: {result.missing}
    """)

In [ ]:
mt.rows().filter(mt.AF > 0.1).select('AF').show()

In [ ]:
mt.filter_rows(mt.locus.contig == "chr1").rows().select("AF").show(10)

In [ ]:
# расчёт частот - 3 версия - возмжно пригодится
def mt_AF_calculated(mt):
    # 1. Подсчёт количества мужчин и женщин (один раз для всей таблицы)
    counts = mt.aggregate_cols(
        hl.struct(
            n_females=hl.agg.count_where(mt.is_female),
            n_males=hl.agg.count_where(~mt.is_female)
        )
    )

    # 2. Обработка аутосом (диплоидный расчёт)
    mt_autosomes = mt.filter_rows(~hl.set(["chrX", "chrY", "chrM"]).contains(mt.locus.contig))
    mt_autosomes = mt_autosomes.annotate_rows(
        AF = hl.agg.mean(mt_autosomes.GT.n_alt_alleles()) / 2
    )

    # 3. Обработка X-хромосомы (оптимизированный вариант)
    mt_x = mt.filter_rows(mt.locus.contig == "chrX")
    
    # Создаём временные переменные для каждой группы
    female_mean = mt_x.filter_cols(mt_x.is_female).annotate_rows(
        temp_f=hl.agg.mean(mt_x.GT.n_alt_alleles()) / 2
    ).rows()

    male_mean = mt_x.filter_cols(~mt_x.is_female).annotate_rows(
        temp_m=hl.agg.mean(mt_x.GT.n_alt_alleles())
    ).rows()

    # Объединяем результаты
    mt_x = mt_x.annotate_rows(
        temp_f = female_mean[mt_x.row_key].temp_f,
        temp_m = male_mean[mt_x.row_key].temp_m
    ).annotate_rows(
        AF = hl.if_else(
            hl.is_defined(mt_x.temp_f) & hl.is_defined(mt_x.temp_m),
            (mt_x.temp_f * 2 * counts.n_females + mt_x.temp_m * counts.n_males) / 
            (2 * counts.n_females + counts.n_males),
            hl.or_else(mt_x.temp_f * 2, mt_x.temp_m)
        )
    ).drop('temp_f', 'temp_m')

    # 4. Обработка гаплоидных регионов (chrM, chrY)
    mt_haploid = mt.filter_rows((mt.locus.contig == "chrM") | (mt.locus.contig == "chrY"))
    mt_haploid = mt_haploid.annotate_rows(
        AF = hl.agg.mean(mt_haploid.GT.n_alt_alleles())
    )

    # 5. Объединение всех частей (предварительно выравниваем схемы)
    return (
        mt_autosomes.select_rows(*mt_autosomes.row_key, 'AF')
        .union_rows(mt_x.select_rows(*mt_x.row_key, 'AF'))
        .union_rows(mt_haploid.select_rows(*mt_haploid.row_key, 'AF'))
    )


In [ ]:
# посмотреть столбцы - РАБОЧИЙ

old_mt = old_mt.cols()
old_mt.show()

new_mt = new_mt.cols()
new_mt.show()